In [1]:
import os

from langchain_ollama import ChatOllama
from langchain_ollama import OllamaEmbeddings

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import PyPDFLoader #ADD!
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
pdf_file_urls = [
    "https://storage.googleapis.com/gg-raggle-public/competitions/29676d73-5675-4278-b1a6-d4a9fdd0a0ba/dataset/Architectural_Design_Service_Contract.pdf",
    "https://storage.googleapis.com/gg-raggle-public/competitions/29676d73-5675-4278-b1a6-d4a9fdd0a0ba/dataset/Call_Center_Operation_Service_Contract.pdf",
    "https://storage.googleapis.com/gg-raggle-public/competitions/29676d73-5675-4278-b1a6-d4a9fdd0a0ba/dataset/Consulting_Service_Contract.pdf",
    "https://storage.googleapis.com/gg-raggle-public/competitions/29676d73-5675-4278-b1a6-d4a9fdd0a0ba/dataset/Content_Production_Service_Contract_(Request_Form).pdf",
    "https://storage.googleapis.com/gg-raggle-public/competitions/29676d73-5675-4278-b1a6-d4a9fdd0a0ba/dataset/Customer_Referral_Contract.pdf",
    "https://storage.googleapis.com/gg-raggle-public/competitions/29676d73-5675-4278-b1a6-d4a9fdd0a0ba/dataset/Draft_Editing_Service_Contract.pdf",
    "https://storage.googleapis.com/gg-raggle-public/competitions/29676d73-5675-4278-b1a6-d4a9fdd0a0ba/dataset/Graphic_Design_Production_Service_Contract.pdf",
    "https://storage.googleapis.com/gg-raggle-public/competitions/29676d73-5675-4278-b1a6-d4a9fdd0a0ba/dataset/M&A_Advisory_Service_Contract_(Preparatory_Committee).pdf",
    "https://storage.googleapis.com/gg-raggle-public/competitions/29676d73-5675-4278-b1a6-d4a9fdd0a0ba/dataset/M&A_Intermediary_Service_Contract_SME_M&A_[Small_and_Medium_Enterprises].pdf",
    "https://storage.googleapis.com/gg-raggle-public/competitions/29676d73-5675-4278-b1a6-d4a9fdd0a0ba/dataset/Manufacturing_Sales_Post-Safety_Management_Contract.pdf",
    "https://storage.googleapis.com/gg-raggle-public/competitions/29676d73-5675-4278-b1a6-d4a9fdd0a0ba/dataset/software_development_outsourcing_contracts.pdf",
    "https://storage.googleapis.com/gg-raggle-public/competitions/29676d73-5675-4278-b1a6-d4a9fdd0a0ba/dataset/Technical_Verification_(PoC)_Contract.pdf",
]

In [6]:
documents = []
for url in pdf_file_urls:
    documents += PyPDFLoader(url).load()
# print(f'type: {type(documents)}')
# documents

type: <class 'list'>


[Document(metadata={'source': 'https://storage.googleapis.com/gg-raggle-public/competitions/29676d73-5675-4278-b1a6-d4a9fdd0a0ba/dataset/Architectural_Design_Service_Contract.pdf', 'page': 0}, page_content=' \n1 \n 建築設計業務委託契約書 \n \nフューチャービルディング株式会社（以下「委託者」という。）とネオファンタジア合同\n会社（以下「受託者」という。）は、委託者による受託者への建築設計業務の委託に関し\nて、以下のとおり建築設計業務委託契約（以下「本契約」という。）を締結する。  \n \n第 1 条 （目的）  \n委託者は、本契約に基づき、【別紙】記載の設計図書の作成及びそれに関連する業務（以\n下「本委託業務」という。）を受託者に委託し、受託者はこれを受託する。 \n \n第 2 条 （業務委託料）  \n1. 委託者は、受託者に対し、本委託業務の対価（以下「業務委託料」という。）として、\n【別紙】記載の報酬を支払う。 \n2. 委託者は、本契約が本契約期間中に終了した場合には、前項の業務委託料に本契約終\n了時までになされた本委託業務の履行割合を乗じた金額を、業務委託料として受託者\nに支払う。ただし、本契約の終了が委託者の責めに帰すべき事由による場合には、委\n託者は前項に定める業務委託料の全額を支払う。 \n \n第 3 条 （支払方法）  \n1. 委託者は、業務委託料を、【別紙】記載の支払時期に、受託者の指定する銀行口座へ\n振込送金の方法により支払う。なお、振込手数料は、委託者の負担とする。 \n2. 委託者が前項の業務委託料の支払を怠った場合、委託者は、支払期限の翌日から完済\nに至るまで年 14.6％の割合による遅延損害金を支払う。 \n \n第 4 条 （業務遂行上の義務等）  \n1. 委託者及び受託者は、本委託業務の遂行につき相互に協力義務を負う。 \n2. 受託者は、本契約に基づき、委託者に納品する成果物に関して、完成義務を負う。 \n3. 受託者は、本委託業務の遂行に関して委託者に適用される法令、監督官庁の告示、

In [21]:
# docの分割設定
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=200,
    chunk_overlap=50,
)
docs = text_splitter.split_documents(documents)

In [22]:
model="llama3.1:latest"
embeddings = OllamaEmbeddings(model=model)
db = DocArrayInMemorySearch.from_documents(docs, embeddings)
retriever = db.as_retriever()

In [9]:
questions = [
    "ソフトウェア開発業務委託契約について、委託料の金額はいくらですか？",
    "グラフィックデザイン制作業務委託契約について、受託者はいつまでに仕様書を作成して委託者の承諾を得る必要がありますか？",
    "コールセンター業務委託契約における請求書の発行プロセスについて、締め日と発行期限を具体的に説明してください。"
]

In [37]:
llm = ChatOllama(model=model)
prompt = ChatPromptTemplate.from_template("""
# ゴール
私は、複数の契約の文章と質問を提供します。
あなたは、どの契約が対象となっているか、そしてその契約の第何条にその質問の回答が
記述されているかを教えて下さい。
# 質問
{question}

# 就業規則
{context}
""")
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | llm | output_parser

In [38]:
question = questions[0]

In [39]:
retriever.invoke(question)

[Document(metadata={'source': 'https://storage.googleapis.com/gg-raggle-public/competitions/29676d73-5675-4278-b1a6-d4a9fdd0a0ba/dataset/Manufacturing_Sales_Post-Safety_Management_Contract.pdf', 'page': 1}, page_content='委託者及び受託者は、本業務に関連する記録について、当該記録を利用しなくなった日か\nら 5 年間保存しなければならない。ただし、関係法令において、それ以上の保存を求めら\nれている記録については、関係法令で定められている期間保存しなければならない。'),
 Document(metadata={'source': 'https://storage.googleapis.com/gg-raggle-public/competitions/29676d73-5675-4278-b1a6-d4a9fdd0a0ba/dataset/M&A_Advisory_Service_Contract_(Preparatory_Committee).pdf', 'page': 2}, page_content='情報を取り扱い、本件業務の目的以外に、これを取り扱ってはならない。  \n3. 委託者及び受託者は、個人情報への不当なアクセス又は個人情報の紛失、盗難、改ざ\nん、漏洩等（以下「漏洩等」という。）の危険に対し、合理的な安全管理措置を講じ\nなければならない。また、委託者及び受託者は、個人情報を、本件業務の遂行のため\nにのみ使用、加工、複写等し、他の目的で使用、加工、複写等してはならない。'),
 Document(metadata={'source': 'https://storage.googleapis.com/gg-raggle-public/competitions/29676d73-5675-4278-b1a6-d4a9fdd0a0ba/dataset/Call_Center_Operation_Service_Contract.pdf', 'page': 1}, page_content='その他の必要情報を書面又は電磁的方法によ

In [40]:
res_1 = chain.invoke(question)
print(question)
print()
print(res_1)

ソフトウェア開発業務委託契約について、委託料の金額はいくらですか？

この質問に答えるには、ソフトウェア開発業務委託契約が何らかの文書の中に記載されているかどうかを調べて、対象の契約の番号とその中での質問に対する回答の場所を知る必要があります。

しかし、そのような文書はリストされていないようです。そのため、質問の内容から推測してみましょう。

ソフトウェア開発業務委託契約についての質問です。 

これらのリストの中には「ソフトウェア開発」に関するものが見当たらないようです。ただし、この質問に答えるために必要な情報は、どれかの契約の中に記載されている可能性があります。

もしもこのリストにあった場合、ご存知のように各文書のメタデータと内容を分析して、第何条にあるかといった回答が可能でしょう。しかし、それを実現するには、そのようなテキストがないようです。
